Calculating total UPDRS with M=5,10,15,. each M is evaluated in separate iteration blocks with their respective result.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from metric_learn import MLKR

In [3]:
import os
path = os.getcwd()
print(path)

f:\AssignmentsUSC\Homework7\homework-7-updated-link-niveditha-7


In [4]:
file_path = path+'\parkinsons_updrs.data'

In [5]:
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
df.head()

,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from metric_learn import MLKR


X = df.drop(columns=['motor_UPDRS', 'total_UPDRS', 'subject#'])
y_total = df['total_UPDRS']

# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_total_train, y_total_test = train_test_split(
    X, y_total, test_size=0.3, random_state=42
)
print("Data split into training and testing sets.")

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data standardized.")
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")


Data split into training and testing sets.
Data standardized.
X_train_scaled shape: (4112, 19)
X_test_scaled shape: (1763, 19)


In [28]:
def transform_with_mlkr(X_train, y_train, X_test, n_components):
    print(f"Applying MLKR with {n_components} components...")
    mlkr = MLKR(init='auto', n_components=n_components)
    mlkr.fit(X_train, y_train)
    X_train_mlkr = mlkr.transform(X_train)
    X_test_mlkr = mlkr.transform(X_test)
    print(f"MLKR transformation done. X_train_mlkr shape: {X_train_mlkr.shape}, X_test_mlkr shape: {X_test_mlkr.shape}")
    return X_train_mlkr, X_test_mlkr


In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge

def tune_krr_hyperparameters(X_train, y_train):
    print("Starting hyperparameter tuning...")
    krr = KernelRidge(kernel='rbf')
    param_grid = {
        'alpha': [0.1, 1, 10],
        'gamma': [0.001, 0.01, 0.1, 1]
    }
    grid_search = GridSearchCV(krr, param_grid, cv=5, scoring='r2', verbose=2, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    print("Hyperparameter tuning completed.")
    print("Best parameters found: ", grid_search.best_params_)
    print("Best R^2 score: ", grid_search.best_score_)
    
    return grid_search.best_estimator_


In [30]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score


kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_total = {}

# Evaluate for different values of M
for M in [5]:
    print(f"\nEvaluating number of components: {M} for Total UPDRS")
    r2_train_scores_total = []
    r2_test_scores_total = []

    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        print(f"\nFold {fold + 1}")
        X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
        y_train_fold_total, y_val_fold_total = y_total_train.iloc[train_index], y_total_train.iloc[val_index]
        
        print(f"Training set size: {X_train_fold.shape[0]}, Validation set size: {X_val_fold.shape[0]}")

        # Transform with MLKR
        X_train_fold_mlkr, X_val_fold_mlkr = transform_with_mlkr(X_train_fold, y_train_fold_total, X_val_fold, n_components=M)

        # Perform hyperparameter tuning
        best_krr = tune_krr_hyperparameters(X_train_fold_mlkr, y_train_fold_total)

        # Train and evaluate the model
        print("Training the model...")
        best_krr.fit(X_train_fold_mlkr, y_train_fold_total)
        y_train_pred_total = best_krr.predict(X_train_fold_mlkr)
        y_val_pred_total = best_krr.predict(X_val_fold_mlkr)
        print("Model training and prediction completed.")

        r2_train_total = r2_score(y_train_fold_total, y_train_pred_total)
        r2_test_total = r2_score(y_val_fold_total, y_val_pred_total)
        print(f"R^2 Train: {r2_train_total}, R^2 Test: {r2_test_total}")

        r2_train_scores_total.append(r2_train_total)
        r2_test_scores_total.append(r2_test_total)

    results_total[M] = {
        'R2_train_mean': np.mean(r2_train_scores_total),
        'R2_test_mean': np.mean(r2_test_scores_total)
    }

print("Total UPDRS Results:")
total_results_df = pd.DataFrame(results_total).T
print(total_results_df)





Evaluating number of components: 5 for Total UPDRS

Fold 1
Training set size: 3289, Validation set size: 823
Applying MLKR with 5 components...
MLKR transformation done. X_train_mlkr shape: (3289, 5), X_test_mlkr shape: (823, 5)
Starting hyperparameter tuning...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Hyperparameter tuning completed.
Best parameters found:  {'alpha': 0.1, 'gamma': 0.001}
Best R^2 score:  0.8289713781067973
Training the model...
Model training and prediction completed.
R^2 Train: 0.8682788985275673, R^2 Test: 0.8386162517536062

Fold 2
Training set size: 3289, Validation set size: 823
Applying MLKR with 5 components...
MLKR transformation done. X_train_mlkr shape: (3289, 5), X_test_mlkr shape: (823, 5)
Starting hyperparameter tuning...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Hyperparameter tuning completed.
Best parameters found:  {'alpha': 0.1, 'gamma': 0.01}
Best R^2 score:  0.8583751044551982
Training the model...
Model trai

Using the above best hyper parameter for next iterations to speed up the process: (code remains same)

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge

def tune_krr_hyperparameters(X_train, y_train):
    print("Starting hyperparameter tuning...")
    krr = KernelRidge(kernel='rbf')
    param_grid = {
        'alpha': [0.1, 1, 10],
        'gamma': [0.001, 0.01, 0.1, 1]
    }
    grid_search = GridSearchCV(krr, param_grid, cv=5, scoring='r2', verbose=2, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    print("Hyperparameter tuning completed.")
    print("Best parameters found: ", grid_search.best_params_)
    print("Best R^2 score: ", grid_search.best_score_)
    
    return grid_search.best_estimator_


In [33]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_total = {}

# Evaluate for different values of M
for M in [10]:
    print(f"\nEvaluating number of components: {M} for Total UPDRS")
    r2_train_scores_total = []
    r2_test_scores_total = []

    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        print(f"\nFold {fold + 1}")
        X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
        y_train_fold_total, y_val_fold_total = y_total_train.iloc[train_index], y_total_train.iloc[val_index]
        
        print(f"Training set size: {X_train_fold.shape[0]}, Validation set size: {X_val_fold.shape[0]}")

        # Transform with MLKR
        X_train_fold_mlkr, X_val_fold_mlkr = transform_with_mlkr(X_train_fold, y_train_fold_total, X_val_fold, n_components=M)

        # Perform hyperparameter tuning
        best_krr = tune_krr_hyperparameters(X_train_fold_mlkr, y_train_fold_total)

        # Train and evaluate the model
        print("Training the model...")
        best_krr.fit(X_train_fold_mlkr, y_train_fold_total)
        y_train_pred_total = best_krr.predict(X_train_fold_mlkr)
        y_val_pred_total = best_krr.predict(X_val_fold_mlkr)
        print("Model training and prediction completed.")

        r2_train_total = r2_score(y_train_fold_total, y_train_pred_total)
        r2_test_total = r2_score(y_val_fold_total, y_val_pred_total)
        print(f"R^2 Train: {r2_train_total}, R^2 Test: {r2_test_total}")

        r2_train_scores_total.append(r2_train_total)
        r2_test_scores_total.append(r2_test_total)

    results_total[M] = {
        'R2_train_mean': np.mean(r2_train_scores_total),
        'R2_test_mean': np.mean(r2_test_scores_total)
    }

print("Total UPDRS Results:")
total_results_df = pd.DataFrame(results_total).T
print(total_results_df)





Evaluating number of components: 10 for Total UPDRS

Fold 1
Training set size: 3289, Validation set size: 823
Applying MLKR with 10 components...
MLKR transformation done. X_train_mlkr shape: (3289, 10), X_test_mlkr shape: (823, 10)
Starting hyperparameter tuning...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Hyperparameter tuning completed.
Best parameters found:  {'alpha': 0.1, 'gamma': 0.01}
Best R^2 score:  0.7712998707466774
Training the model...
Model training and prediction completed.
R^2 Train: 0.9829972083096832, R^2 Test: 0.7972693518402463

Fold 2
Training set size: 3289, Validation set size: 823
Applying MLKR with 10 components...
MLKR transformation done. X_train_mlkr shape: (3289, 10), X_test_mlkr shape: (823, 10)
Starting hyperparameter tuning...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Hyperparameter tuning completed.
Best parameters found:  {'alpha': 0.1, 'gamma': 0.001}
Best R^2 score:  0.7357890728447769
Training the model...
Mod

In [34]:
print("Total UPDRS Results:")
total_results_df = pd.DataFrame(results_total).T
print(total_results_df)

Total UPDRS Results:
    R2_test_mean  R2_train_mean
10      0.796565       0.940495


In [35]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_total_15= {}

# Evaluate for different values of M
for M in [15]:
    print(f"\nEvaluating number of components: {M} for Total UPDRS")
    r2_train_scores_total = []
    r2_test_scores_total = []

    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        print(f"\nFold {fold + 1}")
        X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
        y_train_fold_total, y_val_fold_total = y_total_train.iloc[train_index], y_total_train.iloc[val_index]
        
        print(f"Training set size: {X_train_fold.shape[0]}, Validation set size: {X_val_fold.shape[0]}")

        # Transform with MLKR
        X_train_fold_mlkr, X_val_fold_mlkr = transform_with_mlkr(X_train_fold, y_train_fold_total, X_val_fold, n_components=M)

        # Perform hyperparameter tuning
        best_krr = tune_krr_hyperparameters(X_train_fold_mlkr, y_train_fold_total)

        # Train and evaluate the model
        print("Training the model...")
        best_krr.fit(X_train_fold_mlkr, y_train_fold_total)
        y_train_pred_total = best_krr.predict(X_train_fold_mlkr)
        y_val_pred_total = best_krr.predict(X_val_fold_mlkr)
        print("Model training and prediction completed.")

        r2_train_total = r2_score(y_train_fold_total, y_train_pred_total)
        r2_test_total = r2_score(y_val_fold_total, y_val_pred_total)
        print(f"R^2 Train: {r2_train_total}, R^2 Test: {r2_test_total}")

        r2_train_scores_total.append(r2_train_total)
        r2_test_scores_total.append(r2_test_total)

    results_total_15[M] = {
        'R2_train_mean': np.mean(r2_train_scores_total),
        'R2_test_mean': np.mean(r2_test_scores_total)
    }

print("Total UPDRS Results:")
total_results_df_15 = pd.DataFrame(results_total_15).T
print(total_results_df_15)





Evaluating number of components: 15 for Total UPDRS

Fold 1
Training set size: 3289, Validation set size: 823
Applying MLKR with 15 components...
MLKR transformation done. X_train_mlkr shape: (3289, 15), X_test_mlkr shape: (823, 15)
Starting hyperparameter tuning...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Hyperparameter tuning completed.
Best parameters found:  {'alpha': 0.1, 'gamma': 0.01}
Best R^2 score:  0.7764440719681757
Training the model...
Model training and prediction completed.
R^2 Train: 0.9802953920230464, R^2 Test: 0.7894839881732673

Fold 2
Training set size: 3289, Validation set size: 823
Applying MLKR with 15 components...
MLKR transformation done. X_train_mlkr shape: (3289, 15), X_test_mlkr shape: (823, 15)
Starting hyperparameter tuning...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Hyperparameter tuning completed.
Best parameters found:  {'alpha': 0.1, 'gamma': 0.001}
Best R^2 score:  0.7501330764451435
Training the model...
Mod

In [36]:
print("Total UPDRS Results:")
total_results_df_15 = pd.DataFrame(results_total_15).T
print(total_results_df_15)

Total UPDRS Results:
    R2_test_mean  R2_train_mean
15      0.774247       0.908193


For M=p and total UPDRS 

In [37]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_total_p= {}

# Evaluate for different values of M
for M in [X_train_scaled.shape[1]]:
    print(f"\nEvaluating number of components: {M} for Total UPDRS")
    r2_train_scores_total = []
    r2_test_scores_total = []

    for fold, (train_index, val_index) in enumerate(kf.split(X_train_scaled)):
        print(f"\nFold {fold + 1}")
        X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
        y_train_fold_total, y_val_fold_total = y_total_train.iloc[train_index], y_total_train.iloc[val_index]
        
        print(f"Training set size: {X_train_fold.shape[0]}, Validation set size: {X_val_fold.shape[0]}")

        # Transform with MLKR
        X_train_fold_mlkr, X_val_fold_mlkr = transform_with_mlkr(X_train_fold, y_train_fold_total, X_val_fold, n_components=M)

        # Perform hyperparameter tuning
        best_krr = tune_krr_hyperparameters(X_train_fold_mlkr, y_train_fold_total)

        # Train and evaluate the model
        print("Training the model...")
        best_krr.fit(X_train_fold_mlkr, y_train_fold_total)
        y_train_pred_total = best_krr.predict(X_train_fold_mlkr)
        y_val_pred_total = best_krr.predict(X_val_fold_mlkr)
        print("Model training and prediction completed.")

        r2_train_total = r2_score(y_train_fold_total, y_train_pred_total)
        r2_test_total = r2_score(y_val_fold_total, y_val_pred_total)
        print(f"R^2 Train: {r2_train_total}, R^2 Test: {r2_test_total}")

        r2_train_scores_total.append(r2_train_total)
        r2_test_scores_total.append(r2_test_total)

    results_total_p[M] = {
        'R2_train_mean': np.mean(r2_train_scores_total),
        'R2_test_mean': np.mean(r2_test_scores_total)
    }

print("Total UPDRS Results:")
total_results_df_p = pd.DataFrame(results_total_p).T
print(total_results_df_p)





Evaluating number of components: 19 for Total UPDRS

Fold 1
Training set size: 3289, Validation set size: 823
Applying MLKR with 19 components...
MLKR transformation done. X_train_mlkr shape: (3289, 19), X_test_mlkr shape: (823, 19)
Starting hyperparameter tuning...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Hyperparameter tuning completed.
Best parameters found:  {'alpha': 0.1, 'gamma': 0.001}
Best R^2 score:  0.7467854241980765
Training the model...
Model training and prediction completed.
R^2 Train: 0.807813919831189, R^2 Test: 0.7677421299852951

Fold 2
Training set size: 3289, Validation set size: 823
Applying MLKR with 19 components...
MLKR transformation done. X_train_mlkr shape: (3289, 19), X_test_mlkr shape: (823, 19)
Starting hyperparameter tuning...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Hyperparameter tuning completed.
Best parameters found:  {'alpha': 0.1, 'gamma': 0.001}
Best R^2 score:  0.779050979526171
Training the model...
Mode

Summary for total UPDRS:

Total UPDRS Results for M=5:
   R2_test_mean  R2_train_mean     0.833111        0.92258

Total UPDRS Results for M=10 :
    R2_test_mean  R2_train_mean
    0.796565       0.940495

Total UPDRS Results for M =15:
    R2_test_mean = 0.774247 R2_train_mean = 0.908193

Total UPDRS Results for M =19:
    R2_test_mean = 0.778401   R2_train_mean =         0.887861


NEURAL NETWORKS

Use sklearn's neural network implementation to train a neural network with two
outputs that predicts motor UPDRS and total UPDRS. Use a single layer. You are
responsible to determine other architectural parameters of the network, including
the number of neurons in the hidden and output layers, method of optimization,
type of activation functions, and the L2 \regularization" parameter etc. You
should determine the design parameters via trial and error, by testing your trained
network on the test set and choosing the architecture that yields the smallest test
error. For this part, set early-stopping=False. Remember to standardize your
features. Report your R2 on both training and test sets.

Reference -https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
X = df.drop(columns=['motor_UPDRS', 'total_UPDRS', 'subject#'])
y = df[['motor_UPDRS', 'total_UPDRS']]

# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data split and standardized.")


Data split and standardized.


In [8]:
def train_evaluate_nn(X_train, y_train, X_test, y_test, hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, learning_rate='constant'):
    model = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        alpha=alpha,
        learning_rate=learning_rate,
        max_iter=2000,
        early_stopping=False,
        random_state=42
    )
    model.fit(X_train, y_train)
    print(f"Neural network trained with hidden_layer_sizes={hidden_layer_sizes}, activation={activation}, solver={solver}, alpha={alpha}, learning_rate={learning_rate}")
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    r2_train = r2_score(y_train, y_train_pred, multioutput='raw_values')
    r2_test = r2_score(y_test, y_test_pred, multioutput='raw_values')
    
    print(f"R^2 Train: Motor UPDRS = {r2_train[0]}, Total UPDRS = {r2_train[1]}")
    print(f"R^2 Test: Motor UPDRS = {r2_test[0]}, Total UPDRS = {r2_test[1]}")
    
    return r2_train, r2_test

# Hyperparameter values to try
hidden_layer_sizes = [(50,), (100,), (200,)]
activations = ['relu', 'tanh']
solvers = ['adam', 'sgd']
alphas = [0.0001, 0.001, 0.01]
learning_rates = ['constant', 'adaptive']

best_r2_test = -np.inf
best_params = {}
results = []

# Perform trial and error
for hls in hidden_layer_sizes:
    for act in activations:
        for sol in solvers:
            for alpha in alphas:
                for lr in learning_rates:
                    print(f"\nTrying hidden_layer_sizes={hls}, activation={act}, solver={sol}, alpha={alpha}, learning_rate={lr}")
                    r2_train, r2_test = train_evaluate_nn(X_train_scaled, y_train, X_test_scaled, y_test, hidden_layer_sizes=hls, activation=act, solver=sol, alpha=alpha, learning_rate=lr)
                    results.append({
                        'hidden_layer_sizes': hls,
                        'activation': act,
                        'solver': sol,
                        'alpha': alpha,
                        'learning_rate': lr,
                        'R2_train_motor': r2_train[0],
                        'R2_train_total': r2_train[1],
                        'R2_test_motor': r2_test[0],
                        'R2_test_total': r2_test[1]
                    })
                    if r2_test.mean() > best_r2_test:
                        best_r2_test = r2_test.mean()
                        best_params = {'hidden_layer_sizes': hls, 'activation': act, 'solver': sol, 'alpha': alpha, 'learning_rate': lr}

print(f"Best parameters: {best_params}")
print(f"Best R^2 Test: {best_r2_test}")

# Display results
results_df = pd.DataFrame(results)
print("Trial and Error Results:")
print(results_df)


Trying hidden_layer_sizes=(50,), activation=relu, solver=adam, alpha=0.0001, learning_rate=constant
Neural network trained with hidden_layer_sizes=(50,), activation=relu, solver=adam, alpha=0.0001, learning_rate=constant
R^2 Train: Motor UPDRS = 0.6862427648305214, Total UPDRS = 0.6760005168192673
R^2 Test: Motor UPDRS = 0.6194099218202306, Total UPDRS = 0.614170250929307

Trying hidden_layer_sizes=(50,), activation=relu, solver=adam, alpha=0.0001, learning_rate=adaptive
Neural network trained with hidden_layer_sizes=(50,), activation=relu, solver=adam, alpha=0.0001, learning_rate=adaptive
R^2 Train: Motor UPDRS = 0.6862427648305214, Total UPDRS = 0.6760005168192673
R^2 Test: Motor UPDRS = 0.6194099218202306, Total UPDRS = 0.614170250929307

Trying hidden_layer_sizes=(50,), activation=relu, solver=adam, alpha=0.001, learning_rate=constant
Neural network trained with hidden_layer_sizes=(50,), activation=relu, solver=adam, alpha=0.001, learning_rate=constant
R^2 Train: Motor UPDRS = 0.6

e:\Anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural network trained with hidden_layer_sizes=(100,), activation=tanh, solver=adam, alpha=0.0001, learning_rate=constant
R^2 Train: Motor UPDRS = 0.8950951715291279, Total UPDRS = 0.9003990804031566
R^2 Test: Motor UPDRS = 0.7699043538187633, Total UPDRS = 0.767971047233466

Trying hidden_layer_sizes=(100,), activation=tanh, solver=adam, alpha=0.0001, learning_rate=adaptive


e:\Anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural network trained with hidden_layer_sizes=(100,), activation=tanh, solver=adam, alpha=0.0001, learning_rate=adaptive
R^2 Train: Motor UPDRS = 0.8950951715291279, Total UPDRS = 0.9003990804031566
R^2 Test: Motor UPDRS = 0.7699043538187633, Total UPDRS = 0.767971047233466

Trying hidden_layer_sizes=(100,), activation=tanh, solver=adam, alpha=0.001, learning_rate=constant


...

Neural network trained with hidden_layer_sizes=(100,), activation=tanh, solver=adam, alpha=0.001, learning_rate=adaptive
R^2 Train: Motor UPDRS = 0.8950670736981773, Total UPDRS = 0.9003671201339594
R^2 Test: Motor UPDRS = 0.7699621204779303, Total UPDRS = 0.768020282130004

Trying hidden_layer_sizes=(100,), activation=tanh, solver=adam, alpha=0.01, learning_rate=constant


e:\Anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural network trained with hidden_layer_sizes=(100,), activation=tanh, solver=adam, alpha=0.01, learning_rate=constant
R^2 Train: Motor UPDRS = 0.894826282256355, Total UPDRS = 0.9000847349016857
R^2 Test: Motor UPDRS = 0.7703590055719659, Total UPDRS = 0.7687673473682888

Trying hidden_layer_sizes=(100,), activation=tanh, solver=adam, alpha=0.01, learning_rate=adaptive


e:\Anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural network trained with hidden_layer_sizes=(100,), activation=tanh, solver=adam, alpha=0.01, learning_rate=adaptive
R^2 Train: Motor UPDRS = 0.894826282256355, Total UPDRS = 0.9000847349016857
R^2 Test: Motor UPDRS = 0.7703590055719659, Total UPDRS = 0.7687673473682888

Trying hidden_layer_sizes=(100,), activation=tanh, solver=sgd, alpha=0.0001, learning_rate=constant
Neural network trained with hidden_layer_sizes=(100,), activation=tanh, solver=sgd, alpha=0.0001, learning_rate=constant
R^2 Train: Motor UPDRS = 0.9291567130827659, Total UPDRS = 0.932866332880746
R^2 Test: Motor UPDRS = 0.802569281883293, Total UPDRS = 0.795340421093227

Trying hidden_layer_sizes=(100,), activation=tanh, solver=sgd, alpha=0.0001, learning_rate=adaptive
Neural network trained with hidden_layer_sizes=(100,), activation=tanh, solver=sgd, alpha=0.0001, learning_rate=adaptive
R^2 Train: Motor UPDRS = 0.9304067514631081, Total UPDRS = 0.9342066426197014
R^2 Test: Motor UPDRS = 0.8043294905694349, Total UP

In [10]:
# Display results
results_df = pd.DataFrame(results)
print("Trial and Error Results:")
print(results_df)

Trial and Error Results:
   hidden_layer_sizes activation solver   alpha learning_rate  R2_train_motor  \
0               (50,)       relu   adam  0.0001      constant        0.686243   
1               (50,)       relu   adam  0.0001      adaptive        0.686243   
2               (50,)       relu   adam  0.0010      constant        0.686037   
3               (50,)       relu   adam  0.0010      adaptive        0.686037   
4               (50,)       relu   adam  0.0100      constant        0.688860   
..                ...        ...    ...     ...           ...             ...   
67             (200,)       tanh    sgd  0.0001      adaptive        0.959141   
68             (200,)       tanh    sgd  0.0010      constant        0.957787   
69             (200,)       tanh    sgd  0.0010      adaptive        0.959132   
70             (200,)       tanh    sgd  0.0100      constant        0.957737   
71             (200,)       tanh    sgd  0.0100      adaptive        0.959104   

  

In [11]:
# Sort by R2_test_motor to find the best model for Motor UPDRS
best_motor = results_df.sort_values(by='R2_test_motor', ascending=False).head(1)

# Sort by R2_test_total to find the best model for Total UPDRS
best_total = results_df.sort_values(by='R2_test_total', ascending=False).head(1)

print("Best parameters for Motor UPDRS:")
print(best_motor)

print("\nBest parameters for Total UPDRS:")
print(best_total)

Best parameters for Motor UPDRS:
   hidden_layer_sizes activation solver  alpha learning_rate  R2_train_motor  \
70             (200,)       tanh    sgd   0.01      constant        0.957737   

    R2_train_total  R2_test_motor  R2_test_total  
70        0.962558       0.829763       0.832787  

Best parameters for Total UPDRS:
   hidden_layer_sizes activation solver   alpha learning_rate  R2_train_motor  \
67             (200,)       tanh    sgd  0.0001      adaptive        0.959141   

    R2_train_total  R2_test_motor  R2_test_total  
67        0.964143       0.829182       0.834997  


Early stopping is a technique used to avoid overfitting during training by monitoring the model's performance on a validation set. When the performance on the validation set starts to degrade, training is halted. This helps to find the optimal number of training iterations.

In [16]:
# Best parameters found previously for Motor UPDRS
best_params_motor = {'hidden_layer_sizes': (200,), 'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'learning_rate': 'constant'}

# Train the model with early stopping for Motor UPDRS
model_motor = MLPRegressor(
    hidden_layer_sizes=best_params_motor['hidden_layer_sizes'],
    activation=best_params_motor['activation'],
    solver=best_params_motor['solver'],
    alpha=best_params_motor['alpha'],
    learning_rate=best_params_motor['learning_rate'],
    max_iter=2000,
    early_stopping=True,
    validation_fraction=0.1,
    random_state=42
)

model_motor.fit(X_train_scaled, y_train['motor_UPDRS'])
print("Model for Motor UPDRS trained with early stopping.")

# Predictions for Motor UPDRS
y_train_pred_motor = model_motor.predict(X_train_scaled)
y_test_pred_motor = model_motor.predict(X_test_scaled)

# Evaluate the model for Motor UPDRS
r2_train_motor = r2_score(y_train['motor_UPDRS'], y_train_pred_motor)
r2_test_motor = r2_score(y_test['motor_UPDRS'], y_test_pred_motor)

print(f"Model with Early Stopping R^2 Train (Motor UPDRS): {r2_train_motor}")
print(f"Model with Early Stopping R^2 Test (Motor UPDRS): {r2_test_motor}")

# Best parameters found previously for Total UPDRS
best_params_total = {'hidden_layer_sizes': (200,), 'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'adaptive'}

# Train the model with early stopping for Total UPDRS
model_total = MLPRegressor(
    hidden_layer_sizes=best_params_total['hidden_layer_sizes'],
    activation=best_params_total['activation'],
    solver=best_params_total['solver'],
    alpha=best_params_total['alpha'],
    learning_rate=best_params_total['learning_rate'],
    max_iter=2000,
    early_stopping=True,
    validation_fraction=0.1,
    random_state=42
)

model_total.fit(X_train_scaled, y_train['total_UPDRS'])
print("Model for Total UPDRS trained with early stopping.")

# Predictions for Total UPDRS
y_train_pred_total = model_total.predict(X_train_scaled)
y_test_pred_total = model_total.predict(X_test_scaled)

# Evaluate the model for Total UPDRS
r2_train_total = r2_score(y_train['total_UPDRS'], y_train_pred_total)
r2_test_total = r2_score(y_test['total_UPDRS'], y_test_pred_total)

print(f"Model with Early Stopping R^2 Train (Total UPDRS): {r2_train_total}")
print(f"Model with Early Stopping R^2 Test (Total UPDRS): {r2_test_total}")


Model for Motor UPDRS trained with early stopping.
Model with Early Stopping R^2 Train (Motor UPDRS): 0.7861298431253056
Model with Early Stopping R^2 Test (Motor UPDRS): 0.7294694956844381
Model for Total UPDRS trained with early stopping.
Model with Early Stopping R^2 Train (Total UPDRS): 0.7633195857633778
Model with Early Stopping R^2 Test (Total UPDRS): 0.7019196432712744


comparing with previous network (early stop = false)

Motor UPDRS:

Without early stopping, the model achieves a high R^2 score on both training and test sets, indicating it fits the training data very well and generalizes reasonably to the test data.
With early stopping, the R^2 scores are lower on both training and test sets, suggesting that the model did not overfit to the training data, but also did not generalize as well as in the previous case.
Total UPDRS:

Similar to Motor UPDRS, the model without early stopping achieves higher R^2 scores on both training and test sets compared to the model with early stopping.
The decrease in R^2 scores with early stopping indicates that while the model was prevented from overfitting, it did not perform as well on the test data as the non-early-stopping model.